In [ ]:
# Reload definitions if necessary
using Revise

In [ ]:
using LinearAlgebra
LinearAlgebra.BLAS.set_num_threads(1) # set number of threads for BLAS

using Plots
using LatticeQM
import LatticeQM.Structure.Lattices

# println("Number of workers: ", nworkers())

println("Generate lattice...")
lat = Geometries.honeycomb_twisted(6)
print("Number of sites: ", Lattices.countorbitals(lat), "\n")
Lattices.foldPC!(lat)
sx, sy, sz, sublA, sublB = Operators.getoperator(lat, ["sx", "sy", "sz", "sublatticeAspin", "sublatticeBspin"])
ks = kpath(lat; num_points=80)

println("Generate lattice Hamiltonian...")
hops = Operators.graphene(lat; format=:sparse, mode=:nospin, tz=0.45)
hops = LatticeQM.TightBinding.addspin(hops, :spinhalf)
Operators.addzeeman!(hops, lat, 1e-5) # infinitesimally lift degeneracy
# Operators.addzeeman!(hops, lat, r->sign(r[4]-0.5).*1.5.*[sin(0.0π),0,cos(0.0π)] )

filling = 0.5 + 6.0 / hopdim(hops)
println("Target filling: ", filling)

# Set up interaction
println("Set up meanfield interaction...")
v = Operators.gethubbard(lat; mode=:σx, a=0.5, U=2.6) # interaction potential
ρ_init = Meanfield.initialguess(v, :random; kind=:Z, lat=lat); # initial guess


In [ ]:
println("Create band plot...")
# Get the bands without mean-field terms
ks_plot = kpath(lat; num_points=120)

Operators.setfilling!(hops, filling; nk=9, multimode=:distributed)
bands_plot = getbands(hops, ks_plot, sz; format=:sparse, num_bands=36, multimode=:distributed)

p = plot(bands_plot; colorbar=true)
ylims!(-0.3, 0.3)

In [ ]:
function f!(A, b)
    A.=b
end

# Create a 3x3 matrix
A = [1 2 3; 4 5 6; 7 8 9]
display(A)

# Create a vector
b = [10, 11, 12]

# Update the second column of A with the contents of b
using LinearAlgebra # for the view function
f!(view(A, :, 2), b)

# Display the modified matrix
display(A)


In [ ]:
# using Plots
# p = heatmap(Matrix(real.(hops[[0,0]][1:20, 1:20])); cmap=:RdBu, aspect_ratio=:equal, colorbar=true)#, clims=(-1,1))
# show(p)

# heatmap(Matrix(real.(sz[1:20, 1:20])); cmap=:RdBu, aspect_ratio=:equal, colorbar=true)#, clims=(-1,1))

In [ ]:
# println("Create band plot...")
# # Get the bands without mean-field terms
# ks_plot = kpath(lat; num_points=36)

# # Operators.setfilling!(hops, filling; nk=9, multimode=:distributed)
# bands_plot = getbands(hops, ks_plot, sz; format=:sparse, num_bands=30, multimode=:serial)

# p = plot(bands_plot; colorbar=true)
# ylims!(-0.3, 0.3)